In [6]:
import os
from dotenv import load_dotenv
import pickle

from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser


from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
import faiss

from langchain_community.docstore.in_memory import InMemoryDocstore
from openai import RateLimitError
import time

from langchain.retrievers import BM25Retriever, EnsembleRetriever


from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from flashrank import Ranker

from src.processing.params import (
    RAG_DATA_PATH
)

In [2]:
# # los contents son editables, de cara al contextual rag. Hay que ver si el chunk es editable tb
# test_md_doc = md_docs[0]
# print(test_md_doc)
# # test_md_doc.page_content = "asdfgdfhngfbdsdbfhdgfv"
# # print(test_md_doc)
# text_splitter=RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=50)
# text_splits=text_splitter.split_documents(test_md_doc)
# print(len(text_splits))

In [3]:
RAG_DATA_PATH

'rag_data'

In [2]:
AIDS_DIR = "data"

In [7]:
def get_all_pdfs(dir):
    pdfs = []
    for root, _, files in os.walk(dir):
        for file in files:
            if file.endswith('.pdf'):
                pdfs.append(os.path.join(root, file))
    return pdfs


def get_all_cards(dir):
    markdown_cards = []
    for root, _, files in os.walk(dir):
        for file in files:
            if file.endswith('_card.md'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines = f.readlines()
                    if len(lines) > 5:  # Verifica si el archivo tiene más de 5 líneas
                        markdown_cards.append(file_path)
    return markdown_cards

In [8]:


list_pdf = get_all_pdfs(AIDS_DIR)
list_md = get_all_cards(AIDS_DIR)
print(len(list_pdf), list_pdf)
print(len(list_md), list_md)

29 ['data\\cdti\\cdti-aids\\actuación_conjunta_isciii-cdti\\ficha_apr_mision_salud_2023.pdf', 'data\\cdti\\cdti-aids\\ayudas_cervera_para_centros_tecnológicos\\oobb_cin_644_2023.pdf', 'data\\cdti\\cdti-aids\\ayudas_neotec\\boe-a-2022-6991_oobb.pdf', 'data\\cdti\\cdti-aids\\ayudas_pymes_sello_de_excelencia\\resolucion_sello_2025_web.pdf', 'data\\cdti\\cdti-aids\\ayudas_pymes_sello_de_excelencia\\Sello%20de%20Excelencia%20Convocatoria%202022.pdf', 'data\\cdti\\cdti-aids\\cdti-era-net\\SERA%202022_Texto%20Resoluci%C3%B3n.pdf', 'data\\cdti\\cdti-aids\\cdti-eurostars\\boe-b-2025_1_0.pdf', 'data\\cdti\\cdti-aids\\cdti-partenariados_pilar_2_(sera)\\boe-b-2024-17518_convocatoria_conjunta.pdf', 'data\\cdti\\cdti-aids\\ecosistemas_de_innovación\\boe-a-2023-14674.pdf', 'data\\cdti\\cdti-aids\\innterconecta_-_step\\boe-a-2025-3176_oobb_d.pdf', 'data\\cdti\\cdti-aids\\ipcei_med4cure_convocatoria_de_expresiones_de_interés\\peticion_manifestaciones_de_interes_ipcei_salud.pdf', 'data\\cdti\\cdti-aids\

In [9]:
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment = os.getenv("03_MINI_DEPLOYMENT")  # nombre del *deployment*, NO del modelo
api_version = os.getenv("AZURE_OPENAI_API_VERSION")  # Ajusta según la versión de tu Azure OpenAI

llm = AzureChatOpenAI(
    openai_api_key=api_key,
    azure_endpoint=api_base,
    deployment_name=deployment,
    api_version=api_version,
    # temperature=0
)
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint=api_base,
    api_key=api_key,
    openai_api_version=api_version
)

# Standard RAG

In [6]:
# llms
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment = os.getenv("03_MINI_DEPLOYMENT")  # nombre del *deployment*, NO del modelo
api_version = os.getenv("AZURE_OPENAI_API_VERSION")  # Ajusta según la versión de tu Azure OpenAI

llm = AzureChatOpenAI(
    openai_api_key=api_key,
    azure_endpoint=api_base,
    deployment_name=deployment,
    api_version=api_version,
    # temperature=0
)
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint=api_base,
    api_key=api_key,
    openai_api_version=api_version
)

In [10]:
# Leer ficheros Markdown
markdown_docs = []
for file in list_md:
    loader = TextLoader(file, encoding='utf-8')
    markdown_docs.extend(loader.load())

# Leer ficheros PDF
pdf_docs = []
for file in list_pdf:
    loader = PyPDFLoader(file)
    pdf_docs.extend(loader.load())

# Unir todos los documentos
total_docs = markdown_docs + pdf_docs

In [11]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
text_splits=text_splitter.split_documents(total_docs)
print(len(text_splits))

8271


In [ ]:
# Configuración
documents = text_splits  # Tu lista de Document()s
ids = [str(i + 1) for i in list(range(len(text_splits)))]       # Lista de IDs correspondientes

chunk_processing_size = 200    # Tamaño de lote
sleep_time = 30    # Tiempo de espera entre lotes (segundos)

# Inicializar FAISS
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore= InMemoryDocstore(),
    index_to_docstore_id={}
)

# Procesamiento por lotes
total = len(documents)
for i in range(0, total, chunk_processing_size):
    doc_chunk = documents[i:i + chunk_processing_size]
    id_chunk = ids[i:i + chunk_processing_size]

    success = False
    while not success:
        try:
            vector_store.add_documents(documents=doc_chunk, ids=id_chunk)
            print(f"✅ Lote {i}–{i+len(doc_chunk)-1} añadido.")
            success = True
        except RateLimitError:
            print(f"⚠️ Rate limit alcanzado. Esperando {sleep_time} segundos...")
            time.sleep(sleep_time)
        except Exception as e:
            print(f"❌ Error inesperado en lote {i}–{i+len(doc_chunk)-1}: {e}")
            success = True  # Evita bucle infinito por errores irreparables

    # print(f"⏳ Esperando {sleep_time} segundos antes del siguiente lote...")
    # time.sleep(sleep_time)

print("🎉 Todos los documentos fueron procesados.")

✅ Lote 0–199 añadido.
✅ Lote 200–399 añadido.
✅ Lote 400–599 añadido.
✅ Lote 600–799 añadido.
✅ Lote 800–999 añadido.
✅ Lote 1000–1199 añadido.
✅ Lote 1200–1399 añadido.
✅ Lote 1400–1599 añadido.
✅ Lote 1600–1799 añadido.
✅ Lote 1800–1999 añadido.
✅ Lote 2000–2199 añadido.
✅ Lote 2200–2399 añadido.
✅ Lote 2400–2599 añadido.
✅ Lote 2600–2799 añadido.
✅ Lote 2800–2999 añadido.
✅ Lote 3000–3199 añadido.
✅ Lote 3200–3399 añadido.
✅ Lote 3400–3599 añadido.
✅ Lote 3600–3799 añadido.
✅ Lote 3800–3999 añadido.
✅ Lote 4000–4199 añadido.
✅ Lote 4200–4399 añadido.
✅ Lote 4400–4599 añadido.
✅ Lote 4600–4799 añadido.
✅ Lote 4800–4999 añadido.
✅ Lote 5000–5199 añadido.
✅ Lote 5200–5399 añadido.
✅ Lote 5400–5599 añadido.
✅ Lote 5600–5799 añadido.
✅ Lote 5800–5999 añadido.
✅ Lote 6000–6199 añadido.
✅ Lote 6200–6399 añadido.
✅ Lote 6400–6599 añadido.
✅ Lote 6600–6799 añadido.
✅ Lote 6800–6999 añadido.
✅ Lote 7000–7199 añadido.
✅ Lote 7200–7399 añadido.
✅ Lote 7400–7599 añadido.
✅ Lote 7600–7799 añadido

In [37]:
VECTORSTORE_PATH = "rag_data/faiss/vanilla_vectorstore"

# vectorstore = FAISS.from_documents(text_splits, embeddings)

vector_store.save_local(VECTORSTORE_PATH)





In [12]:
# local BM25 retriever

def save_bm25_retriever(text_splits, path):

    with open(path, "wb") as f:
        pickle.dump(text_splits, f)

def load_bm25_retriever(path, k = 5):
    with open(path, "rb") as f:
        bm25_data = pickle.load(f)
    keyword_retriever = BM25Retriever.from_documents(bm25_data)
    keyword_retriever.k =  k

    return keyword_retriever





BM25_PATH = "rag_data/bm25/vanilla_bm25.pkl"

save_bm25_retriever(text_splits, BM25_PATH)



In [13]:
keyword_retriever = load_bm25_retriever(BM25_PATH)

In [14]:
keyword_retriever

BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x0000020C5D071130>, k=5)

In [4]:
VECTORSTORE_PATH = "rag_data/faiss/vanilla_vectorstore"
vector_store = FAISS.load_local(VECTORSTORE_PATH, embeddings, allow_dangerous_deserialization=True)

# vanilla RAG


In [16]:
template = """
Use the following pieces of context to answer the question at the end.
If you don't know the answer, say that you don't know.
Always answer in spanish.
Context: {context}
Question: {question}
"""
# Return the sources at the end of the answer, in markdown format, with the name of the document and the page number.

retriever = vector_store.as_retriever(search_kwargs={"k": 10})

prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

query = "Dame información sobre la ayuda 'proyectos de i+d'"
response = chain.invoke(query)
print(response)

La ayuda para proyectos de I+D está destinada a empresas que desarrollen actividades de investigación y desarrollo aplicadas, orientadas a la creación o mejora significativa de procesos productivos, productos o servicios. Entre sus características principales se encuentran:

1. Los proyectos deben demostrar un diferencial tecnológico, es decir, aportar elementos innovadores que los diferencien de las tecnologías existentes en el mercado.

2. Pueden comprender tanto actividades de investigación industrial como de desarrollo experimental.

3. La ayuda está abierta a proyectos presentados individualmente por una única empresa, aunque también se pueden presentar en consorcio. En el caso de proyectos de cooperación nacional, si participan varias empresas, deben cumplir ciertos requisitos (por ejemplo, dentro del consorcio, al menos una debe ser una pyme y, en algunos casos, se exige autonomía entre las entidades participantes).

4. No existen restricciones en cuanto al sector o a la tecnolo

# RAG with sources

In [ ]:


# # Paso 1: Prompt con placeholders
# template = """
# Usa los siguientes fragmentos de contexto para responder la pregunta al final.
# Si no sabes la respuesta, di que no la sabes.
# Siempre responde en español.

# Contexto:
# {context}

# Pregunta: {question}

# Devuelve las fuentes al final de la respuesta, en formato markdown, con el nombre del documento y número de página si están disponibles.
# """
# prompt = ChatPromptTemplate.from_template(template)


# def get_retrieved_docs(query):
#     return vector_store.similarity_search(query, k=5)


# def serialize_docs(retrieved_docs):
#     sources = ""
#     for doc in retrieved_docs:
#         source = doc.metadata['source']
#         try:
#             page = doc.metadata['page']
#         except KeyError:
#             page = None
#         new_source = f"Source: {source}, Page: {page}\n" if page else f"Source: {source}\n"
#         sources += new_source

#     return sources


# retrieval_chain = (
#     {
#         "question": RunnablePassthrough(), # query
#         "context": RunnablePassthrough()
#             | RunnableLambda(get_retrieved_docs) # similarity_search
#             | RunnableLambda(serialize_docs) # sources
#     }
#     | prompt
#     | llm
#     | StrOutputParser()
# )


# query = "Dame información sobre la ayuda 'proyectos de i+d'"
# response = retrieval_chain.invoke(query)
# print(response)

La ayuda “proyectos de I+D” está dirigida a financiar iniciativas de investigación, desarrollo e innovación (I+D+i) que impulsen la competitividad y la capacidad tecnológica de las empresas y centros de investigación. Su objetivo es apoyar proyectos que aborden aspectos tecnológicos innovadores, permitiendo desde la fase de investigación básica hasta el desarrollo aplicado y la innovación de producto o proceso.

Entre sus características se incluyen:

• Financiación destinada a cubrir parte de los costes del proyecto, pudiendo variar según el tamaño de la empresa, el sector o la línea temática de la convocatoria.  
• Requisitos técnicos y administrativos que garantizan la viabilidad y el carácter innovador de la propuesta, por lo que es necesario presentar una documentación específica que justifique el potencial impacto tecnológico y económico del proyecto.  
• Posibles líneas de convocatoria diferenciadas según sectores (por ejemplo, aeroespacial o salud), lo que implica que, en cada 

# opcion de sources por fuera





In [18]:
template = """
Usa los siguientes fragmentos de contexto para responder la pregunta al final.
Si no sabes la respuesta, di que no la sabes.
Siempre responde en español.

Contexto:
{context}

Pregunta: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

def get_retrieved_docs(query):
    # return vector_store.similarity_search(query, k=5)
    return retriever.get_relevant_documents(query)


def serialize_docs(retrieved_docs):
    sources = "Fuentes:\n"
    for doc in retrieved_docs:
        source = doc.metadata['source']
        try:
            page = doc.metadata['page']
        except KeyError:
            page = None
        new_source = f"* {source}, Página: {page}\n" if page else f"* {source}\n"
        sources += new_source

    return sources

def get_sources(query):
    retrieved_docs = vector_store.similarity_search(query, k=5)
    sources = serialize_docs(retrieved_docs)
    return sources


query = "Dame información sobre la ayuda 'proyectos de i+d'"

response = chain.invoke(query)
sources = get_sources(query)
print(response + "\n\n" + sources)


La ayuda para “proyectos de I+D” está dirigida a financiar proyectos empresariales de investigación y desarrollo con carácter aplicado. A continuación, te resumo los aspectos principales extraídos de los documentos de contexto:

1. Objetivo de la ayuda:  
   • Financia proyectos destinados a la creación o mejora significativa de procesos productivos, productos o servicios.  
   • Se valoran actividades de investigación industrial y de desarrollo experimental.

2. Características de los proyectos:  
   • Deben demostrar un aspecto tecnológico diferencial respecto a las tecnologías existentes en el mercado.  
   • Son proyectos presentados por una única empresa, entendiendo que, en el caso de pymes, se aplican los criterios establecidos en la normativa correspondiente.

3. Beneficiarios y convocatoria:  
   • Los beneficiarios son empresas sin restricciones en cuanto al sector o tecnología a desarrollar.  
   • La convocatoria es abierta durante todo el año, permitiendo la presentación d

# hybrid search

In [156]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
retrieved_docs = retriever.get_relevant_documents(query)
retrieved_docs
# sources = serialize_docs(retrieved_docs)
# print(sources)

[Document(id='6947', metadata={'producer': 'iTextSharp 5.1.2 (c) 1T3XT BVBA', 'creator': 'PyPDF', 'creationdate': '2025-03-04T11:02:53+01:00', 'moddate': '2025-03-04T11:02:53+01:00', 'source': 'data\\cdti\\cdti-aids\\proyectos_de_i_+_d\\proyectos_de_id.pdf', 'total_pages': 42, 'page': 1, 'page_label': '2'}, page_content='REF.: Ficha proyectos de I+D \nFecha: 02/01/2015 \nÚltima modificación \nTexto principal: 27/02/2025 y Anexos: 29/11/2024 \n \n2 \n1. Definición.  \nLos proyectos de I+D son proyectos de investigación y desarrollo empresarial de \ncarácter aplicado para la creación o mejora significativa de un proceso \nproductivo, producto o servicio. Los proyectos deben demostrar un aspecto \ntecnológico diferencial sobre las tecnologías existentes en el mercado.'),
 Document(id='404', metadata={'source': 'data\\cdti\\cdti-aids\\proyectos_de_i_+_d\\proyectos_de_i_+_d_card.md'}, page_content='# Actuación\nProyectos CDTI de I+D\n\n# Objetivo general de la actuación\nAyudas a proyectos 

In [ ]:

keyword_retriever = BM25Retriever.from_documents(text_splits)
keyword_retriever.k =  5
bm25_retrieved_docs = keyword_retriever.get_relevant_documents(query)
bm25_retrieved_docs

[Document(metadata={'producer': 'Antenna House PDF Output Library 6.6.1477 (Linux64)', 'creator': 'eBOE', 'creationdate': '2023-06-19T19:00:09+01:00', 'keywords': 'ORDEN CIN/644/2023 de 16/06/2023;MINISTERIO DE CIENCIA E INNOVACIÓN;BOE-A-2023-14674;BOE 146 de 2023;14674;20/06/2023', 'moddate': '2023-06-19T19:19:44+02:00', 'trapped': '/False', 'subject': 'BOE-A-2023-14674', 'author': 'MINISTERIO DE CIENCIA E INNOVACIÓN', 'title': 'Disposición 14674 del BOE núm. 146 de 2023', 'source': 'data\\cdti\\cdti-aids\\ayudas_cervera_para_centros_tecnológicos\\oobb_cin_644_2023.pdf', 'total_pages': 40, 'page': 28, 'page_label': '29'}, page_content='hábiles desde que se formule la solicitud de información será motivo de exclusión del \nprocedimiento. A efectos de información y control de estas ayudas, se cederá la \ninformación entre los sistemas establecidos para la gestión y control de estas \nsubvenciones con los Sistemas de los Fondos Europeos.\n5.\u2003La comprobación formal para la liquidació

In [ ]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, keyword_retriever],
    weights=[0.5, 0.5]
)

ensemble_retriever.get_relevant_documents(query)

[Document(metadata={'producer': 'Antenna House PDF Output Library 6.6.1477 (Linux64)', 'creator': 'eBOE', 'creationdate': '2023-06-19T19:00:09+01:00', 'keywords': 'ORDEN CIN/644/2023 de 16/06/2023;MINISTERIO DE CIENCIA E INNOVACIÓN;BOE-A-2023-14674;BOE 146 de 2023;14674;20/06/2023', 'moddate': '2023-06-19T19:19:44+02:00', 'trapped': '/False', 'subject': 'BOE-A-2023-14674', 'author': 'MINISTERIO DE CIENCIA E INNOVACIÓN', 'title': 'Disposición 14674 del BOE núm. 146 de 2023', 'source': 'data\\cdti\\cdti-aids\\ayudas_cervera_para_centros_tecnológicos\\oobb_cin_644_2023.pdf', 'total_pages': 40, 'page': 28, 'page_label': '29'}, page_content='hábiles desde que se formule la solicitud de información será motivo de exclusión del \nprocedimiento. A efectos de información y control de estas ayudas, se cederá la \ninformación entre los sistemas establecidos para la gestión y control de estas \nsubvenciones con los Sistemas de los Fondos Europeos.\n5.\u2003La comprobación formal para la liquidació

In [ ]:



template = """
Usa los siguientes fragmentos de contexto para responder la pregunta al final.
Si no sabes la respuesta, di que no la sabes.
Siempre responde en español y usando la sintaxis de markdown, pero no devuelvas la respuesta envuelta en un bloque markdown.

Contexto:
{context}

Pregunta: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

retriever_vectordb = vector_store.as_retriever(search_kwargs={"k": 5})

keyword_retriever = BM25Retriever.from_documents(text_splits)
keyword_retriever.k =  5

ensemble_retriever = EnsembleRetriever(
    retrievers=[
        retriever_vectordb,
        keyword_retriever
    ],
    weights=[0.5, 0.5]
)

prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

chain = (
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

def get_retrieved_docs(query):
    # return vector_store.similarity_search(query, k=5)
    return ensemble_retriever.get_relevant_documents(query)


def serialize_docs(retrieved_docs):
    sources = "Fuentes:\n"
    for doc in retrieved_docs:
        source = doc.metadata['source']
        try:
            page = doc.metadata['page']
        except KeyError:
            page = None
        new_source = f"* {source}, Página: {page}\n" if page else f"* {source}\n"
        sources += new_source

    return sources

def get_sources(query):
    retrieved_docs = vector_store.similarity_search(query, k=5)
    sources = serialize_docs(retrieved_docs)
    return sources


query = "Dame información sobre la ayuda 'proyectos de i+d'"

response = chain.invoke(query)
sources = get_sources(query)
print(response + "\n\n" + sources)

# Ayuda para Proyectos de I+D

La ayuda para proyectos de I+D está orientada a apoyar a empresas en el desarrollo de proyectos de investigación y desarrollo empresarial, destinados a lograr una **creación o mejora significativa** de procesos productivos, productos o servicios.

## Características generales

- **Ámbito de aplicación:**  
  Se financian proyectos que impliquen actividades de investigación industrial y/o desarrollo experimental. No existe restricción en cuanto al sector o tecnología a desarrollar.

- **Objetivo:**  
  Impulsar el desarrollo tecnológico diferencial, ofreciendo apoyo para la realización de estudios y mejoras que aporten un aspecto innovador en relación con las tecnologías existentes en el mercado.

- **Tipo de proyectos:**  
  Los proyectos deben demostrar un claro componente de innovación o mejora respecto al estado actual de la tecnología. Pueden ser tanto proyectos de investigación aplicada como de desarrollo experimental.

- **Beneficiarios:**  
  Son e

# reranknig


In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)



In [5]:


template = """
Usa los siguientes fragmentos de contexto para responder la pregunta al final.
SSi no sabes la respuesta, di que no la sabes.
Siempre responde en español y usando la sintaxis de markdown, pero no devuelvas la respuesta envuelta en un bloque markdown.

Contexto:
{context}

Pregunta: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

retriever_vectordb = vector_store.as_retriever(search_kwargs={"k": 5})

keyword_retriever = BM25Retriever.from_documents(text_splits)
keyword_retriever.k =  5

ensemble_retriever = EnsembleRetriever(
    retrievers=[
        retriever_vectordb,
        keyword_retriever
    ],
    weights=[0.5, 0.5]
)

ranker = Ranker()
compressor = FlashrankRerank()


# compressor = SentenceTransformerRerank(model="cross-encoder/ms-marco-MiniLM-L-6-v2")

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, 
    base_retriever=ensemble_retriever
)


prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

chain = (
    {"context": compression_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

def get_retrieved_docs(query):
    # return vector_store.similarity_search(query, k=5)
    return compression_retriever.get_relevant_documents(query)


def serialize_docs(retrieved_docs):
    sources = "Fuentes:\n"
    for doc in retrieved_docs:
        source = doc.metadata['source']
        try:
            page = doc.metadata['page']
        except KeyError:
            page = None
        new_source = f"* {source}, Página: {page}\n" if page else f"* {source}\n"
        sources += new_source

    return sources

def get_sources(query):
    retrieved_docs = vector_store.similarity_search(query, k=5)
    sources = serialize_docs(retrieved_docs)
    return sources


query = "Dame información sobre la ayuda 'proyectos de i+d'"

response = chain.invoke(query)
sources = get_sources(query)
print(response + "\n\n" + sources)

NameError: name 'text_splits' is not defined

# test SQLiteDB

In [1]:
from src.processing.sql_db import SQLiteDB
import pandas as pd

sqlite_db = SQLiteDB()
sqlite_db.sqlite_setup()

query = "SELECT count(*) FROM aids_table"
df_sql = pd.read_sql(query, con=sqlite_db.engine)
df_sql.head()


loading data/processing/cdti_table.csv
SQLite DB saved in: rag_data/sqlite/aids_table.sqlite


,count(*)
0,32


In [5]:
import pandas as pd

from src.processing.utils import get_aid_list
from src.processing.cdti.loaders import CDTILoader
from src.processing.cdti.params import DATA_PATH

In [ ]:
table_dir = "data/processing"
filename = "cdti_table.csv"


In [7]:
RAG_DATA_PATH = "rag_data"
sqlite_path = f"{RAG_DATA_PATH}/sqlite/aids_table.sqlite"

In [8]:
from sqlalchemy import create_engine

In [11]:
df = pd.read_csv(f"{CSV_PATH}/{filename}")
df.head(1)

,organismo,nombre,linea,fecha_inicio,fecha_fin,objetivo,beneficiarios,anio,area,presupuesto_minimo,...,intensidad_de_subvencion,intensidad_del_prestamo,tipo_financiacion,forma_y_plazo_de_cobro,minimis,region_de_aplicacion,tipo_de_consorcio,costes_elegibles,link_ficha_tecnica,link_convocatoria
0,CDTI,Proyectos de I + D,Proyectos de I+D individuales,Todo el año,Todo el año,Ayudas a proyectos de I+D desarrollados por em...,Empresas.,2025,I+D,175.000 euros,...,• Investigación industrial \n – Intensidad ge...,Ayuda de hasta el 85% del presupuesto aprobado.,Ayuda parcialmente reembolsable.,"Formas de cobro:\n• Pago anualizado, compuesto...",Si,"Sí, la convocatoria impone restricciones geogr...",El consorcio debe estar formado por un mínimo ...,• Gastos directos:\n – Gastos de personal: in...,https://www.cdti.es/node/115,https://www.cdti.es/sites/default/files/2025-0...


In [ ]:
engine = create_engine(f'sqlite:///{sqlite_path}')
df.to_sql('tabla', con=engine, if_exists='replace', index=False)

print(f"Archivo SQLite guardado en: {sqlite_path}")

In [ ]:
from sqlalchemy import create_engine

# Cargar el archivo CSV en un DataFrame
csv_path = 'ruta_al_archivo.csv'  # Reemplaza con la ruta de tu archivo CSV
df = pd.read_csv(csv_path)

# Guardar el DataFrame como una base de datos SQLite usando sqlalchemy
sqlite_path = os.path.join(DATA_PATH, 'archivo.sqlite')  # Usa DATA_PATH definido previamente
engine = create_engine(f'sqlite:///{sqlite_path}')
df.to_sql('tabla', con=engine, if_exists='replace', index=False)

print(f"Archivo SQLite guardado en: {sqlite_path}")


In [ ]:
from langchain.chains import SQLDatabaseChain
from langchain.sql_database import SQLDatabase
from langchain.llms import OpenAI

# Crear una conexión a la base de datos SQLite
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_path}")

# Crear un modelo LLM (puedes ajustar los parámetros según tus necesidades)
llm = OpenAI(temperature=0)

# Crear el SQLDatabaseChain
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

print("SQLDatabaseChain creado con éxito.")